<a href="https://colab.research.google.com/github/jasminl/adaptics/blob/master/onnx_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install onnx
!pip install onnxruntime
!pip install onnxruntime_extensions
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.7 MB/s eta 0:00:00


In [1]:
from onnx import helper, onnx_pb as onnx_proto, save_model
import onnxruntime as _ort
from onnxruntime_extensions import (
    onnx_op, PyCustomOpDef, make_onnx_model,
    get_library_path as _get_library_path)


nodes = [helper.make_node('CustomOp', ['input'], ['output_1'], domain='ai.onnx.contrib'),
         helper.make_node('Abs', ['output_1'], ['output_2'])]

@onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    return np.round(f)

input0 = helper.make_tensor_value_info(
    'input', onnx_proto.TensorProto.FLOAT, [3, 5])
output1 = helper.make_tensor_value_info(
    'output_1', onnx_proto.TensorProto.FLOAT, [3, 5])
output2 = helper.make_tensor_value_info(
    'output_2', onnx_proto.TensorProto.FLOAT, [3, 5])

graph = helper.make_graph(nodes, 'test0', [input0], [output2])
model = helper.make_model(
    graph, opset_imports=[helper.make_operatorsetid('ai.onnx.contrib', 1)], ir_version=7)
save_model(model, 'mymodel.onnx')


In [4]:
from onnx import load_model
import onnxruntime as _ort
from onnxruntime_extensions import (
    onnx_op, PyCustomOpDef, make_onnx_model,
    get_library_path as _get_library_path)
import numpy as np

so = _ort.SessionOptions()

# Define a custom operator: this must be defined before the call to register_custom_ops_library
@onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    return np.round(f)

so.register_custom_ops_library(_get_library_path())  # Must first call any @onnx_op before this

new_model = load_model('mymodel.onnx')
print(new_model)
sess = _ort.InferenceSession(new_model.SerializeToString(), so)

res = sess.run(None, {'input': np.random.rand(3, 5).astype(np.float32)})
print(res)

ir_version: 7
graph {
  node {
    input: "input"
    output: "output_1"
    op_type: "CustomOp"
    domain: "ai.onnx.contrib"
  }
  node {
    input: "output_1"
    output: "output_2"
    op_type: "Abs"
  }
  name: "test0"
  input {
    name: "input"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
  output {
    name: "output_2"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
}
opset_import {
  domain: "ai.onnx.contrib"
  version: 1
}



Fail: [ONNXRuntimeError] : 1 : FAIL : Failed to add kernel for CustomOp ai.onnx.contrib CPUExecutionProvider: Conflicting with a registered kernel with op versions.

In [17]:
import json
import io
import onnx
from onnx import load_model
new_model = load_model('mymodel.onnx')

print(new_model.opset_import)
print(new_model)
# #output = io.StringIO()
# with open('tmp.txt', 'w') as f:
#     print(new_model, file=f)
# #contents = output.getvalue()
# #output.close()
# with open('tmp.txt', 'r') as f:
#     onnx.parser.parse_model(f.read())

[domain: "ai.onnx.contrib"
version: 1
]
ir_version: 7
graph {
  node {
    input: "input"
    output: "output_1"
    op_type: "CustomOp"
    domain: "ai.onnx.contrib2"
  }
  node {
    input: "output_1"
    output: "output_2"
    op_type: "Abs"
  }
  name: "test0"
  input {
    name: "input"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
  output {
    name: "output_2"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
}
opset_import {
  domain: "ai.onnx.contrib"
  version: 1
}

